<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Dodanie_top_typ_filmu/colabs/Dane_z_dodatkowymi_kolumnami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1293, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 1293 (delta 341), reused 217 (delta 192), pack-reused 835
Receiving objects: 100% (1293/1293), 5.94 MiB | 6.22 MiB/s, done.
Resolving deltas: 100% (743/743), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
# !git checkout <nazwa_twojego_brancha>

In [4]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 200 kB 46.3 MB/s 
     |████████████████████████████████| 199 kB 14.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=5c1e8602a33aa33b14c7d2db8da1f2ae96cc2be6080dbb36e1c7dc3f8338b9fe
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-29 17:41:37--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop2.tgz’

spark-3.3.1-bin-had 100%[===============

In [5]:
import pandas as pd
from lib.pyspark_init import create_spark_context
from lib.adding_columns import add_all_columns
from lib.const import DATA_WITH_AWARDS, TRAIN_DATA, VALIDATE_DATA, TEST_DATA

In [6]:
spark = create_spark_context()
spark

In [7]:
!chmod +x download_data.sh
!./download_data.sh

--2022-12-29 17:41:54--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.35.24.74, 13.35.24.122, 13.35.24.56, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.35.24.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238526258 (227M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 227.48M  21.8MB/s    in 12s     

2022-12-29 17:42:07 (19.1 MB/s) - ‘name.basics.tsv.gz’ saved [238526258/238526258]

--2022-12-29 17:42:07--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.35.24.74, 13.35.24.122, 13.35.24.56, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.35.24.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291823241 (278M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 278.30M  22.0MB/s   

In [8]:
%%time
data = spark.read.parquet(DATA_WITH_AWARDS)
data.show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-------------+------------+------------+-------------+--------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|        primaryName|      knownForTitles|         year_oscars|     category_oscars|         film_oscars|       winner_oscars|         year_globes|     category_globes|         film_globes|          win_globes|year_emmy|category_emmy|nominee_emmy|company_emmy|producer_emmy|win_emmy|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-

In [9]:
%%time
data = add_all_columns(spark, data)
#data.show()

CPU times: user 117 ms, sys: 20.9 ms, total: 138 ms
Wall time: 4.84 s


## Dodanie top type dla aktora

In [10]:
from pyspark.sql.functions import explode, split
from pyspark.sql import Window
from pyspark.sql import Window
from pyspark.sql.functions import count, row_number, desc, first

In [12]:
df2 = data.select('nconst', explode(data.titleType).alias('titleType'))
df3 = df2.select('nconst', explode(split(df2.titleType, ',')).alias('titleType'))

w = Window.partitionBy('nconst', 'titleType')
aggregated_table = df3.withColumn("count",count("*").over(w)).withColumn("rn",
                                                                 row_number().over(
                                                                     w.orderBy(desc("count")))).filter(
                                                                         "rn = 1").groupBy('nconst').agg(
                                                                             first('titleType').alias('top_type'))
aggregated_table.show()

+---------+--------+
|   nconst|top_type|
+---------+--------+
|nm0000004|   movie|
|nm0000012|   movie|
|nm0000021|   movie|
|nm0000023|   movie|
|nm0000029|   movie|
|nm0000086|   movie|
|nm0000100|   movie|
|nm0000113|   movie|
|nm0000131|   movie|
|nm0000161|   movie|
|nm0000174|   movie|
|nm0000179|   movie|
|nm0000194|   movie|
|nm0000198|   movie|
|nm0000199|   movie|
|nm0000237|   movie|
|nm0000240|   movie|
|nm0000301|   movie|
|nm0000304|   movie|
|nm0000339|   movie|
+---------+--------+
only showing top 20 rows



In [13]:
data = data.join(aggregated_table, data.nconst==aggregated_table.nconst, 'left').drop(aggregated_table.nconst)
data.show()

+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-------------+------------+------------+-------------+--------+---------------------+---------+---------------------+---------+-------------------+-------+--------+--------------------+---------+---------+
|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|         year_oscars|     category_oscars|         film_oscars|       winner_oscars|         year_globes|     category_globes|         film_globes|          win_globes|year_emmy|category_emmy|nominee_emmy|company_emmy|produc

## Podział zbioru

In [ ]:
%%time
train_data, validate_data, test_data = data.randomSplit([0.7, 0.1, 0.2], seed=1)

In [ ]:
%%time
data.write.parquet(TRAIN_DATA)

In [ ]:
%%time
data.write.parquet(VALIDATE_DATA)

In [ ]:
%%time
data.write.parquet(TEST_DATA)